In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [3]:
annot_f <- '/oak/stanford/groups/mrivas/ukbb24983/exome/annotation/20201025_exome_oqfe_2020/ukb24983_exomeOQFE.annotation.20201217.compact.tsv.gz'
dup_f   <- '/oak/stanford/groups/mrivas/ukbb24983/exome/qc/oqfe_2020/intermediate_files/ukb24983_exomeOQFE.duplicates.tsv.gz'


In [4]:
dup_f %>% fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') -> dup_df


In [5]:
annot_f %>%
fread(
    colClasses = c('#CHROM'='character'),
    select=c('#CHROM', 'POS', 'ID', 'REF', 'ALT', 'FILTER', 'f_miss', 'UKB_white_british_hwe_p')
) %>% rename('CHROM'='#CHROM') %>%
mutate(
    QC_duplicated = ! (ID %in% (dup_df$ID)),
    QC_sample_miss = (f_miss < .1),
    QC_WB_HWE_p = (log10(UKB_white_british_hwe_p)>-15)
) -> annot_df


In [8]:
annot_df %>%
count(FILTER)

FILTER,n
<chr>,<int>
.,17777950


In [13]:
annot_df %>%
mutate(FILTER = paste0(
    FILTER,
    if_else(QC_duplicated  & (!is.na(QC_duplicated)),  '', ';duplicated'),
    if_else(QC_sample_miss & (!is.na(QC_sample_miss)), '', ';sample_miss'),
    if_else(QC_WB_HWE_p    & (!is.na(QC_WB_HWE_p)),    '', ';WB_HWE_p')
)) %>%
mutate(FILTER = str_replace(FILTER, '^.;', '')) %>%
count(FILTER)


FILTER,n
<chr>,<int>
.,17582030
duplicated,91
duplicated;sample_miss,146
duplicated;sample_miss;WB_HWE_p,76
duplicated;WB_HWE_p,13
sample_miss,150824
sample_miss;WB_HWE_p,20165
WB_HWE_p,24605


This number (17,582,030 variants) matches with what we have in : `/oak/stanford/groups/mrivas/ukbb24983/exome/qc/oqfe_2020/ukb24983_exomeOQFE.passQC.20201222.tsv.gz`

